# More Basics of Pandas

Last session we introduced Pandas quickly. Now we review the basics of using Pandas, with a more interactive workflow to solidify understanding and facility with this powerful Python toolkit.

## Reviewing Pandas Basics

In [5]:
from pandas import DataFrame
import pandas as pd

In [ ]:
import pandas as pd
df = pd.read_csv('DEC_10_SF1_P1_with_ann.csv')


### Basics of indexing in Pandas

The first use of indexing is to use a slice, just like we have done with other Python objects. Below we slice the first 5 index values of the first dimension of the dataframe.

In [ ]:
df[:5]


The first indexing method is equivalent to usinf the iloc indexing method, which uses the integer based indexing, purely based on the location of the index.

In [ ]:
df.iloc[:5]

A second way to index is using loc, which uses the labels of the index. Note that this approach includes the second value in the index range, whereas iloc does not.

In [ ]:
df.loc[:5]

Note that indexing can work for both rows and colums

In [ ]:
df.loc[:5, : 'Geography']

In [ ]:
df.iloc[:5, :3]

We can select rows based on their value as well.  Notice that we nest df[df[condition]] to get this result.

In [ ]:
df['Population'] < 200

In [ ]:
df[df['Population'] < 200]

In [ ]:
df['Id2']==21015980100

In [ ]:
df[df['Id2']==21015980100]

Here we show how to set a value of a cell in the table, identifying a specific row by index label, and setting its population, in this case to a None value, which Pandas interprets as a NaN (missing value).

In [ ]:
df.loc[688,'Population'] = None

We can filter for values that are Null

In [ ]:
df[df['Population'].isnull()]

Or more commonly, filter out the null values.

In [ ]:
df[df['Population'].notnull()]

Here we find and print records that are in Fayette County, using the str attribute and 'contains' to search for the county name in geodisplay.

In [ ]:
df[df['Geography'].str.contains('Fayette County')]

We can find the unique values of a column (not very interesting in this particular case)

We saw last time how to use the str attribute to do string manipulation, such as to create a new column.  We need to explore some more advanced string processing, but let's use a smaller dataframe for that.

In [ ]:
df['state'] = df['Geography'].str.split(',').str[2]
df[:5]

In [ ]:
df['county'] = df['Geography'].str.split(',').str[1]
df[:5]

In [ ]:
df['county'].unique()

And count how many times each unique value is in the data

In [ ]:
df['county'].value_counts()

### More complex use of Indexing and String Manipulation -- Cleaning the Bedroom Field in the Craigslist data

In [25]:
import pandas as pd
import numpy as np
cl = pd.read_csv('items.csv')
cl.head()

,neighborhood,title,price,bedrooms,pid,longitude,date,link,latitude,sqft,sourcepage
0,(SOMA / south beach),"1bed + Den, 1bath at Mission Bay",$2895,/ 1br - 950ft² -,4046628359,-122.399663,Sep 4 2013,/sfc/apa/4046628359.html,37.774623,/ 1br - 950ft² -,http://sfbay.craigslist.org/sfc/apa/
1,(SOMA / south beach),Love where you live!,$3354,/ 1br - 710ft² -,4046761563,NaN,Sep 4 2013,/sfc/apa/4046761563.html,NaN,/ 1br - 710ft² -,http://sfbay.craigslist.org/sfc/apa/
2,(inner sunset / UCSF),We Welcome Your Furry Friends! Call Today!,$2865,/ 1br - 644ft² -,4046661504,-122.470727,Sep 4 2013,/sfc/apa/4046661504.html,37.765739,/ 1br - 644ft² -,http://sfbay.craigslist.org/sfc/apa/
3,(financial district),Golden Gateway Commons | 2BR + office townhous...,$5500,/ 2br - 1450ft² -,4036170429,NaN,Sep 4 2013,/sfc/apa/4036170429.html,NaN,/ 2br - 1450ft² -,http://sfbay.craigslist.org/sfc/apa/
4,(lower nob hill),Experience Luxury Living in San Fransisco!,$3892,/ 2br -,4046732678,NaN,Sep 4 2013,/sfc/apa/4046732678.html,NaN,/ 2br -,http://sfbay.craigslist.org/sfc/apa/


This is a worked part of your assignment for this week.  Here is a code snippet that uses some of what we just learned above, and extends it to clean the bedroom field in this data.  Adapting it to clean sqft remains for you to do in the assignment...

Below are two different ways to do this.  The first involves looping over the index of the dataframe, and finding the beginning and end of the substring we are looking for, to isolate the bedrooms value.

In [27]:
for label in cl['bedrooms'].index:
    if isinstance(cl['bedrooms'][label], str) and not pd.isnull(cl['bedrooms'][label]):
        end = cl['bedrooms'][label].find('br')
        if end == -1:
            cl.loc[label,'bedrooms'] = np.nan
        else:
            start = cl['bedrooms'][label].find('/ ') + 2
            cl.loc[label,'bedrooms'] = int(cl['bedrooms'][label][start: end])
cl['bedrooms'][:9]

0      1
1      1
2      1
3      2
4      2
5      1
6    NaN
7      1
8      2
Name: bedrooms, dtype: object

In [26]:
for label in cl['sqft'].index:
    if isinstance(cl['sqft'][label], str) and not pd.isnull(cl['sqft'][label]):
        end = cl['sqft'][label].find('ft')
        if end == -1:
            cl.loc[label,'sqft'] = np.nan
        else:
            start = cl['sqft'][label].find('- ') + 2
            cl.loc[label,'sqft'] = (cl['sqft'][label][start: end])
cl['sqft'][:9]

0     950
1     710
2     644
3    1450
4     NaN
5     NaN
6        
7     781
8     NaN
Name: sqft, dtype: object

Here is a second way to do this that is much more elegant, and in some ways much simpler since it does not require looping over the index values.  It uses the apply() method to apply an element-wise function.  This is covered briefly on page 133 of Pandas for Data Analysis, and more exensively in online documentation.

In [36]:
# first we create a method that works on a single instance
def clean_br(value1):
    ''' value - string from Craigslist entry that includes size of bedrooms
        returns - integer size of bedrooms
    '''
    if isinstance(value1, str):
        end = value1.find('ft')
        if end == -1:
            return None
        else:
            start = value.find('-') + 2
            return int(value1[start:end])
cl['sqft'][:9]
        

0     950
1     710
2     644
3    1450
4     NaN
5     NaN
6        
7     781
8     NaN
Name: sqft, dtype: object

In [35]:
# first we create a method that works on a single instance
def clean_br(value):
    ''' value1 - string from Craigslist entry that includes number of 
        returns - integer number of bedrooms
    '''
    if isinstance(value, str):
        end = value.find('br')
        if end == -1:
            return None
        else:
            start = value.find('/') + 2
            return int(value[start:end])
cl['bedrooms'][:9]
        

0      1
1      1
2      1
3      2
4      2
5      1
6    NaN
7      1
8      2
Name: bedrooms, dtype: object

In [ ]:
# for example, we can apply it to a single string
clean_br('/ 1br - 950ft² -')

In [ ]:
# and another
clean_br('/ 2br - 1450ft² -')

In [ ]:
# really what we want to do is to apply it to all rows in a dataframe
# we can do this with a loop, similar to what we have above
cl = pd.read_csv('items.csv')

for i, row in cl.iterrows(): 
    row['bedrooms'] = clean_br(row['bedrooms'])
    

In [ ]:
# Let's look at the result.  We see that it doesn't change.  What went wrong?
# let's go back and fix it together
cl['bedrooms'][:9]

In [ ]:
# if we have a very simple function, we can define it all in one line using the keyword lambda

cl['bedrooms_plus2'] = cl['bedrooms'].apply(lambda x : x+2)
cl['bedrooms_plus2'][:9]


In [ ]:
# the above line is equivalent to:
def plus_two(x):
    return x+2

cl['bedrooms_plus2'] = cl['bedrooms'].apply(plus_two)
cl['bedrooms_plus2'][:9]

## Working with Pandas -- an in-class working session

OK, let's create a DataFrame from a dictionary, following the example on pg 116 of Python for Data Analysis (PDA).

In [3]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'year': [2000, 2001, 2002, 2001, 2002],
'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}


Explain the contents and structure of 'data'

What does 'DataFrame(data)' do? What if we did not begin that line with 'df ='?

Look at the contents of df, using just df by itself, and 'print df'.  

In [7]:
df1=pd.DataFrame(data)

df1

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


We can refer to a column in two ways:

In [ ]:
df1.loc[:4, :'state']

In [ ]:
df1.state

We can step through the rows in a dataframe

In [ ]:
for i in df1.state.index:
    print(df1.state[i])

And find the index value within each entry of a specific substring

In [ ]:
for i in df1.state.index:
    print(df1.state[i].find('io'))

In [ ]:
for i in df1.state.index:
    if df1.state[i]=='Ohio':
        print(df1.state[i])
    else:
        print('Missing')

## Now Your Turn

Below are a series of questions, with the answers remaining for you to fill in by using pandas expressions that draw on the methods in Chapter 5.  You should not need to use anything more than the content of this chapter -- a subset of the methods summarized above, to do this exercise.  Hopefully you can complete it within class if you've been keeping up with the reading.

How can we get a quick statistical profile of all the numeric columns?

In [ ]:
df1.describe()

Can you get a profile of a column that is not numeric, like state? Try it.

In [ ]:
df1['state'].describe()

How can we print the data types of each column?

In [ ]:
df1.dtypes

How can we print just the column containing state names?

In [ ]:
df1.state

How can we get a list of the states in the DataFrame, without duplicates?

In [ ]:
df1['state'].drop_duplicates()

How can we get a count of how many rows we have in each state?

In [ ]:
df1['state'].count()

How can we compute the mean of population across all the rows?

In [ ]:
df1['pop'].mean()

How can we compute the maximum population across all the rows?

In [ ]:
df1['pop'].max()

How can we compute the 20th percentile value of population? 

In [ ]:
df1['pop'].quantile(.2)

How can we compute a Boolean array indicating whether the state is 'Ohio'?

In [ ]:
for item in df1.state.index:
    if df1.state[item]== "Ohio":
        print(True)
    else:
        print(False)

How can we select and print just the rows for Ohio?

In [18]:
df1[df1['state']=='Ohio']


,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6


How can we create a new DataFrame containing only the Ohio records?

In [20]:
dfo = df1[df1['state']=='Ohio']
dfo

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6


How can we select and print just the rows in which population is more than 2?

In [ ]:
df1[df1['pop']>=2]

How could we compute the mean of population that is in Ohio, averaging across years?

In [ ]:
df1[df1['state']=='Ohio'].mean()

How can we print the DataFrame, sorted by State and within State, by Population?

How can we print the row for Ohio, 2002, selecting on its values (not on row and column indexes)?

In [24]:
print(dfo.loc[2])
    

state    Ohio
year     2002
pop       3.6
Name: 2, dtype: object


How can we use row and column indexing to set the population of Ohio in 2002 to 3.4?

In [ ]:
df1.loc[2,'pop'] = 3.4
df1

How can we use row and column indexing to append a new record for Utah, initially with no population or year? 

In [ ]:
new_row = {'state': 'Utah'}
df2=df1.append(new_row, ignore_index=True)
df2

How can we set the population to 2.5 and year to 2001 for the new record?

In [ ]:
df2.loc[5,'year']= 2001
df2.loc[5, 'pop']= 2.5
df2